In [1]:
import tensorflow as tf 
import numpy as np

In [2]:
# 加载数据集：只用训练集即可
(train_image, train_label), _ = tf.keras.datasets.mnist.load_data()  # 测试集用个占位符就行，不用它！

In [3]:
# 维度拓展，从数组转为图像：
train_image = tf.expand_dims(train_image, -1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [4]:
# 转换数据类型：
train_image = tf.cast( train_image/255, tf.float32 )  # 归一化后，转为float32
train_label = tf.cast( train_label, tf.int32 )

In [5]:
# tf.data进行数据集输入：
dataset = tf.data.Dataset.from_tensor_slices( (train_image,train_label) )
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int32)>

In [6]:
# 数据集乱序：
dataset = dataset.shuffle(60000).repeat().batch(64)

# 自定义搭建

还是各种用tf.keras模块下的高阶函数：
- tf.keras.Sequential()：顺序模型
- tf.keras.layers.xxx：各种层的搭建
- tf.keras.optimizers.xxx：各种实例化（可自定义）优化器
- tf.keras.losses.xxx：各种实例化（可自定义）损失函数
- tf.keras.metrics：汇总的指标计算模块

### 1. 网络搭建、一些重要的查看：

In [7]:
model = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 16, [3,3], activation = 'relu', input_shape = (None, None, 1) ),  # input_shape这样写即任何输入尺寸都行！
    tf.keras.layers.Conv2D( 32, [3,3], activation = 'relu' ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)   #  其实可以不用softmax激活到[0,1]概率值的，哪个分数最高就是哪个！ 
] )

In [8]:
# 查看网络有哪些可训练参数：那些数值就是对它们的随机初始化值！
model.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[ 1.29310027e-01, -5.01299798e-02,  9.21515375e-02,
            1.89935759e-01,  1.64683297e-01,  1.10040650e-01,
           -2.74499804e-02, -7.71654248e-02,  1.95985988e-01,
            1.54879496e-01, -1.62694275e-01,  1.89048663e-01,
            9.53941196e-02,  3.10119838e-02,  1.41150907e-01,
            7.41352886e-02]],
 
         [[ 6.04042858e-02,  1.32942870e-01,  1.32009387e-04,
           -1.44048512e-01,  1.48379013e-01,  7.47742802e-02,
            3.91086936e-02,  1.67300567e-01, -1.77160487e-01,
           -1.58046350e-01,  8.45053941e-02,  1.79753050e-01,
            8.37582797e-02,  3.01231891e-02,  1.57819524e-01,
            4.60865349e-02]],
 
         [[ 1.08924493e-01, -1.71941027e-01, -5.51987290e-02,
            1.77584127e-01,  1.09551325e-01,  1.77902922e-01,
           -1.67748064e-01, -1.12429403e-01,  7.08381385e-02,
           -1.64635479e-03, -9.06915069e-02,  1.63559392

In [9]:
# 自定义优化器：
optimizer = tf.keras.optimizers.Adam( lr = 0.001 )

In [10]:
# 自定义损失函数：里面小写的那些要额外指定y_true和y_pred两个参数，大写的直接就是可调用对象没多余参数！
loss_func = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True )  # 前面Dense没有用softmax激活，这样要告诉一下！

In [11]:
# 在Eager模式下，dataset本身就是一个可迭代对象：单位是一个元组（图像,标签）
# 取出来第一个元素（第一批次）看看：
(features, labels) = next( iter(dataset) )

In [12]:
features.shape, labels.shape
# dataset已经按批次分割了，所以单位是一个批次，每个批次里有64张图：

(TensorShape([64, 28, 28, 1]), TensorShape([64]))

In [13]:
# 当前模型已经可以做处理了！虽然参数没有训练，但已经都给了随机初始值！
# 放一个批次图像进去，每一张图都会给一个10分类的结果！
prediction = model( features )
prediction.shape  # 64张图，每张图给一个10分类的结果！

TensorShape([64, 10])

In [14]:
# 查看经过网络之后的10分类结果，哪个值是最大的（就选哪个），并对比真实labels —— 现在完全可以，只是参数还都没训练好！
tf.argmax( prediction, axis = 1 ), labels 

(<tf.Tensor: id=150, shape=(64,), dtype=int64, numpy=
 array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       dtype=int64)>, <tf.Tensor: id=132, shape=(64,), dtype=int32, numpy=
 array([2, 6, 3, 9, 9, 2, 7, 7, 1, 4, 1, 3, 8, 3, 4, 3, 5, 9, 2, 0, 5, 0,
        4, 3, 0, 6, 2, 6, 6, 3, 5, 9, 1, 5, 1, 8, 5, 1, 6, 0, 4, 1, 8, 1,
        3, 3, 7, 6, 0, 9, 3, 9, 1, 1, 5, 0, 4, 2, 3, 5, 1, 2, 5, 6])>)

### 2. 损失函数、训练函数自定义：

In [15]:
# 定义损失函数：
# 功能：输入模型、训练图像、对应真实标签；计算网络对该图像的预测值；返回预测值与真实值间的损失
def loss(model, images, labels_real):
    labels_predict = model(images)  # 网络的预测
    return loss_func(labels_real, labels_predict)  # 返回二者的交叉熵损失

In [16]:
# 每个epoch的计算与优化函数：
def train_step(model, images, labels_real):
    with tf.GradientTape() as t:  # 跟踪loss_step函数的梯度运算过程！
        loss_step = loss(model, images, labels_real)  # 每个batch的损失函数：目标函数！！
    grads = t.gradient( loss_step, model.trainable_variables )  # 求目标函数关于“各个”可训练参数的梯度值！
    optimizer.apply_gradients( zip(grads, model.trainable_variables) )  # 用上一步得到“各个”可训练参数梯度值，修改“各个”可训练参数

In [17]:
# 总体训练的执行函数：主函数！
def train():
    for epoch in range(10):
        print('进入批次训练！')
        for (batch, (images, labels_real)) in enumerate(dataset):  # enumerate就是再多加一个序号，刚好作为batch号！
            train_step( model, images, labels_real )  # 每个batch的训练！
        print( 'Epoch{}，第{}批次训练完毕'.format(epoch+1, batch) )

In [ ]:
train()

### 3. tf.keras.metrics汇总“指标”计算模块

tf.keras.metrics最常用的就是求各种“**均值**”指标：因为自定义训练的数据的单位都是batch批次，所以例如求loss、acc的均值，都指的是求一个batch内所有图片的loss均值和acc均值！—— 当然，求一个epoch的loss、acc均值也是可以的！

下面是一个使用案例：生成一个**可调用对象**！每次把新的数据给这个对象用即可。

总结及注意：
- m.result().numpy()：查看当前状态（数值）
- m.reset_states()：状态重置 —— 如果生成的可调用对象m不重置，它会一直记录当前的状态！
- tf.keras.metrics.SparseCategoricalAccuracy()：顺序编码交叉熵的准确率计算

In [18]:
m = tf.keras.metrics.Mean('acc_mean')  # 生成一个可调用对象m，括号里的字符串只是起一个别名而已（打印的时候打这个名字）！

In [19]:
m(10)

<tf.Tensor: id=173, shape=(), dtype=float32, numpy=10.0>

In [20]:
# 每新给一个数值，就求该数值和前面所有数值的均值：
m(20)  # (20 + 10）/2 = 15

<tf.Tensor: id=182, shape=(), dtype=float32, numpy=15.0>

In [21]:
# 显示结果：面向对象的展示！
m.result().numpy()

15.0

In [24]:
# 可以直接把一个list给这个“可调用对象m”使用：
m( [20, 30, 40] )
m.result().numpy()  # (15 + 20 + 30 + 40) / 4 = 26.25

26.25

In [25]:
# 重置可调用对象m：
m.reset_states()

In [28]:
m(10)

<tf.Tensor: id=235, shape=(), dtype=float32, numpy=10.0>

In [35]:
# 求顺序编码交叉熵的准确率：
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')  # 创建一个可调用对象

In [36]:
(images, labels_real) = next( iter(dataset) )
lables_predict = model(images)

In [37]:
a( labels_real, lables_predict )  # 预测一个batch，64张图的模型预测准确率

<tf.Tensor: id=350, shape=(), dtype=float32, numpy=0.078125>

In [38]:
tf.argmax( lables_predict, axis = 1 ), labels_real 

(<tf.Tensor: id=352, shape=(64,), dtype=int64, numpy=
 array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
       dtype=int64)>, <tf.Tensor: id=321, shape=(64,), dtype=int32, numpy=
 array([1, 6, 0, 4, 2, 1, 8, 8, 0, 1, 2, 3, 0, 4, 1, 3, 5, 9, 7, 1, 4, 1,
        1, 3, 4, 6, 8, 3, 1, 2, 6, 3, 7, 4, 5, 6, 6, 8, 2, 7, 8, 4, 3, 9,
        0, 6, 8, 1, 2, 9, 0, 0, 8, 2, 5, 5, 1, 6, 4, 5, 7, 8, 4, 2])>)

In [39]:
5/64

0.078125